# Demo: Play Before/After Audio Files

This notebook demonstrates how to load and compare paired audio recordings:
- **Before (tape-input)**: Original audio before tape recording
- **After (tape-output-recordings)**: Audio after being recorded and played back through the tape deck

Listen to the differences introduced by the analog tape recording process!

## 1. Imports and Configuration

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path for imports
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))

import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment

from src.utils import Config

In [ ]:
# Load configuration
cfg = Config()
cfg.print_paths()

## 2. Load Dataset Charlie

Dataset Charlie contains paired before/after recordings of frequency sweeps and noise.

In [ ]:
# Dataset paths
DATASET_DIR = cfg.get_datasets_dir() / "dataset-charlie"
BEFORE_DIR = DATASET_DIR / "tape-input"
AFTER_DIR = DATASET_DIR / "tape-output-recordings"

print(f"Dataset directory: {DATASET_DIR}")
print(f"Before (input):    {BEFORE_DIR}")
print(f"After (output):    {AFTER_DIR}")

# List available recording categories
print("\nAvailable recording categories:")
categories = sorted([d.name for d in BEFORE_DIR.iterdir() if d.is_dir()])
for cat in categories:
    print(f"  - {cat}")

## 3. Utility Functions

In [ ]:
def find_paired_files(before_dir: Path, after_dir: Path) -> list[tuple[Path, Path]]:
    """Find matching before/after file pairs.
    
    Matches files by replacing 'vorher' with 'taped' in filenames.
    
    Args:
        before_dir: Directory containing 'before' files
        after_dir: Directory containing 'after' files
    
    Returns:
        List of (before_path, after_path) tuples
    """
    pairs = []
    
    for before_file in sorted(before_dir.rglob('*.wav')):
        # Convert before filename to after filename
        # e.g., "sweep 5sec V1 2024-03-31 vorher.wav" -> "sweep 5sec V1 2024-03-31 taped.wav"
        after_name = before_file.name.replace('vorher', 'taped')
        
        # Find corresponding after file in same relative subdirectory
        rel_path = before_file.parent.relative_to(before_dir)
        after_file = after_dir / rel_path / after_name
        
        if after_file.exists():
            pairs.append((before_file, after_file))
    
    return pairs


def display_audio_comparison(before_file: Path, after_file: Path, title: str = None):
    """Display side-by-side comparison of before/after audio.
    
    Args:
        before_file: Path to before audio file
        after_file: Path to after audio file
        title: Optional title for the comparison
    """
    # Load audio
    y_before, sr_before = librosa.load(str(before_file))
    y_after, sr_after = librosa.load(str(after_file))
    
    # Get file info
    before_pydub = AudioSegment.from_file(str(before_file))
    after_pydub = AudioSegment.from_file(str(after_file))
    
    if title:
        print(f"\n{'='*60}")
        print(f"{title}")
        print(f"{'='*60}")
    
    print(f"\nBefore: {before_file.name}")
    print(f"  Duration: {len(before_pydub)/1000:.2f}s | Sample rate: {before_pydub.frame_rate} Hz | Peak: {before_pydub.max_dBFS:.1f} dBFS")
    
    print(f"\nAfter:  {after_file.name}")
    print(f"  Duration: {len(after_pydub)/1000:.2f}s | Sample rate: {after_pydub.frame_rate} Hz | Peak: {after_pydub.max_dBFS:.1f} dBFS")
    
    # Plot waveforms
    fig, axes = plt.subplots(2, 1, figsize=(14, 6))
    
    axes[0].set_title("BEFORE (Original)")
    librosa.display.waveshow(y_before, sr=sr_before, ax=axes[0], color='cyan', alpha=0.7)
    axes[0].set_ylabel("Amplitude")
    
    axes[1].set_title("AFTER (Tape Recorded)")
    librosa.display.waveshow(y_after, sr=sr_after, ax=axes[1], color='orange', alpha=0.7)
    axes[1].set_ylabel("Amplitude")
    axes[1].set_xlabel("Time (s)")
    
    plt.tight_layout()
    plt.show()
    
    # Play audio
    print("\n▶ BEFORE (Original):")
    ipd.display(ipd.Audio(str(before_file)))
    
    print("\n▶ AFTER (Tape Recorded):")
    ipd.display(ipd.Audio(str(after_file)))

## 4. Find All Paired Files

In [ ]:
# Find all paired files
pairs = find_paired_files(BEFORE_DIR, AFTER_DIR)

print(f"Found {len(pairs)} before/after pairs:\n")
for before, after in pairs:
    rel_path = before.relative_to(BEFORE_DIR)
    print(f"  {rel_path}")

## 5. Compare Before/After Recordings

Listen to the difference the tape recording process makes!

In [ ]:
# Compare first pair: Short frequency sweep
if len(pairs) > 0:
    before, after = pairs[0]
    display_audio_comparison(before, after, title="Frequency Sweep Comparison")

In [ ]:
# Compare another pair
if len(pairs) > 4:
    before, after = pairs[4]
    display_audio_comparison(before, after, title="Another Recording Comparison")

## 6. Spectrogram Comparison

Visualize the frequency content differences between before and after.

In [ ]:
def display_spectrogram_comparison(before_file: Path, after_file: Path, title: str = None):
    """Display spectrogram comparison of before/after audio."""
    
    # Load audio
    y_before, sr_before = librosa.load(str(before_file))
    y_after, sr_after = librosa.load(str(after_file))
    
    # Compute spectrograms
    D_before = librosa.amplitude_to_db(np.abs(librosa.stft(y_before)), ref=np.max)
    D_after = librosa.amplitude_to_db(np.abs(librosa.stft(y_after)), ref=np.max)
    
    if title:
        print(f"\n{title}")
        print(f"{'='*60}")
    
    # Plot spectrograms
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    img1 = librosa.display.specshow(D_before, sr=sr_before, x_axis='time', y_axis='hz', ax=axes[0])
    axes[0].set_title("BEFORE (Original) - Spectrogram")
    fig.colorbar(img1, ax=axes[0], format='%+2.0f dB')
    
    img2 = librosa.display.specshow(D_after, sr=sr_after, x_axis='time', y_axis='hz', ax=axes[1])
    axes[1].set_title("AFTER (Tape Recorded) - Spectrogram")
    fig.colorbar(img2, ax=axes[1], format='%+2.0f dB')
    
    plt.tight_layout()
    plt.show()


# Show spectrogram comparison for a sweep file
if len(pairs) > 0:
    before, after = pairs[0]
    display_spectrogram_comparison(before, after, title="Spectrogram: Before vs After Tape Recording")

## 7. Browse All Pairs

Select a pair index to compare any recording.

In [ ]:
# Change this index to browse different pairs (0 to len(pairs)-1)
PAIR_INDEX = 0

if PAIR_INDEX < len(pairs):
    before, after = pairs[PAIR_INDEX]
    display_audio_comparison(before, after, title=f"Pair {PAIR_INDEX}: {before.parent.name}")
else:
    print(f"Invalid index. Choose 0 to {len(pairs)-1}")